# Hierarchical Partial-Order (HPO) Model Simulation

This document provides a comprehensive guide to simulating data from a **Hierarchical Partial-Order (HPO) Model**. The HPO model is designed to generate hierarchical structures of partial orders based on latent variables influenced by multiple assessors. This simulation accounts for global latent scores, assessor-specific deviations, and observed ranking data within specified choice sets.

1: Global Latent U:U(0)∼N(0,Σ ρ​)
2: Assessor-Specific Latent Scores
3:Transformation to Preference Scores
4.Generation of Partial Orders for each η(a)

Discussion with Geoff abou this script:
1. The hierachical partial order work 
	a. will yi contain a full rank for partial order or only 
	b. talk about the data structure 
	c. when we do inference, what are we inferreing to? How to generate the choisen set 

In [57]:
import numpy as np
import math
from scipy.stats import multivariate_normal, norm
import networkx as nx
import random
import seaborn as sns
import pandas as pd
from collections import Counter, defaultdict
import itertools
import matplotlib.pyplot as plt
from typing import List, Optional, Dict, Any
import sys as sys

# Make sure these paths and imports match your local project structure
sys.path.append('../src/utils')  # Example path
from po_fun import BasicUtils, StatisticalUtils#
from mallow_function import Mallows
#from po_accelerator import LogLikelihoodCache
from typing import Dict, List

# Transform Eta change the G function 
# The initial setting is not correct, we need to have the O_i variable , and then we could generate the hierachical relationship out

# Then deploy the partial order relationship 

# Peform the MCMC simulation alogrithem and think the method about how to write it 

In [58]:

def G_inv(p):
    # Gumbel quantile function
    # Safeguard tiny p to avoid log(0)
    eps = 1e-15
    p_safe = np.clip(p, eps, 1 - eps)
    return -np.log(-np.log(p_safe))

In [59]:
M0 = [0, 1, 2, 3, 4, 5] ## This is the total list 
assessors = [1, 2, 3]
# (Sometimes called the "leaf" sets M_a in the partial-order hierarchy.)
M_a_list = [
    [0, 2, 4],   # M_1
    [1, 2, 3],   # M_2
    [1, 3, 4]    # M_3
]

# For each assessor a, the list of choice sets O_{a,i}
# Meaning: in each "task" or "batch", assessor a was asked to rank this subset.
# We'll store them in a dictionary keyed by assessor, 
# each value is a list of subsets (choice sets).
O_a_i_dict = {
    1: [
        [0, 2, 4],
        [0, 2]
    ],
    2: [
        [1, 2,5],
        [1, 3],
        [1,4,5]
    ],
    3: [
        [1, 3],
        [1, 3, 4],
        [2,4,5]
    ]
}

# Observed orders y_{a,i} (one for each choice set above).
# For instance, y_{1,0} is an order of [0,4,2] (lowest rank first or highest first, depending on convention).
# We'll store these in the same dictionary shape.
# These are "real" or "observed" orders for the subsets O_{a,i}.
y_a_i_dict = {
    1: [
        [0, 4, 2],   # observed order for subset [0,2,4]
        [0, 2]       # observed order for subset [0,2]
    ],
    2: [
        [1, 2],      # observed order for subset [1,2],will yi contain a full rank for partial order 
        [3, 1]       # observed order for subset [1,3]
        # [1, 4]       # observed order for subset [1,4,5]
    ],
    3: [
        [3, 1],      # observed order for subset [1,3]
        [4, 3, 1],   # observed order for subset [1,3,4]
        [5,4,2]
    ]
}

alpha = np.array([0.5, -0.2, 0.3, 0.1, 0.0, 1.2]) # The characteristic vector for the object, which is a score , alpha is beta*X , alpha is the global variable and need M0 dimension 

In [60]:
K = 2 
rho = 0.9 # Should follow a beta distributino, let's assume this first 
tau = 0.8# should follow a beta distributino, let's assume this first 

In [61]:
def simulate_HPO_with_tasks(
    M0: List[int],
    assessors: List[int],
    M_a_list: List[List[int]],
    O_a_i_dict: Dict[int, List[List[int]]],
    alpha: np.ndarray,            # Must be non-None and length = len(M0)
    K: int,
    rho: float,
    tau: float,
    random_seed: int = 42
):
 
    # ----------------------------
    # 1) Setup: random seed & covariance
    # ----------------------------
    rng = np.random.default_rng(random_seed)  # single global RNG
    random.seed(random_seed)                  # for any other random usage

    # Build the correlation matrix Sigma_rho (compound-symmetric)
    Sigma_rho = np.full((K, K), rho, dtype=float)
    np.fill_diagonal(Sigma_rho, 1.0)

    # Validate alpha length
    n_global = len(M0)
    if len(alpha) != n_global:
        raise ValueError(
            f"alpha must have length {n_global}, but got length {len(alpha)}"
        )

    # ----------------------------
    # 2) Global Latent U^(0)
    # ----------------------------
    mvn_global = multivariate_normal(mean=np.zeros(K), cov=Sigma_rho)
    U0 = mvn_global.rvs(size=n_global)  # shape: (n_global, K)

    # ----------------------------
    # 3) Assessor-level Latents
    # ----------------------------
    U_a_list = []
    eta_a_list = []
    h_a_list = []
    h_a_i = {a: {} for a in assessors}  # partial orders restricted to tasks

    for idx, a in enumerate(assessors):
        # Items for this assessor:
        Ma = M_a_list[idx]
        n_a = len(Ma)

        # For j in M_a: U^(a)_j ~ N( tau*U^(0)_j, (1 - tau^2)*Sigma_rho )
        # We do it based on each object j in this question.
        Ua = np.zeros((n_a, K), dtype=float)
        for i_loc, j_global in enumerate(Ma):
            mean_vec = tau * U0[j_global, :]
            cov_mat  = (1 - tau**2) * Sigma_rho
            Ua[i_loc, :] = rng.multivariate_normal(mean=mean_vec, cov=cov_mat)
        U_a_list.append(Ua)

        # Transform to Gumbel-based eta^(a)
        eta_a = np.zeros_like(Ua)
        for i_loc, j_global in enumerate(Ma):
            # step 1: Gaussian -> Uniform(0,1)
            p_vec = norm.cdf(Ua[i_loc, :])
            # step 2: Uniform(0,1) -> Gumbel -> + alpha
            gumbel_vec = np.array([G_inv(p) for p in p_vec])
            eta_a[i_loc, :] = gumbel_vec + alpha[j_global]
        eta_a_list.append(eta_a)

        # partial order h^(a)
        h_a = BasicUtils.generate_partial_order(eta_a)
        h_a_list.append(h_a)

        # Restrict partial order to each choice set O_{a,i}
        O_list = O_a_i_dict.get(a, [])
        for i_task, subset in enumerate(O_list):
            subset_indices_in_Ma = []
            for item_j in subset:
                if item_j in Ma:
                    local_idx = Ma.index(item_j)
                    subset_indices_in_Ma.append(local_idx)

            sub_size = len(subset_indices_in_Ma)
            h_sub = np.zeros((sub_size, sub_size), dtype=int)
            for r, i_loc_ in enumerate(subset_indices_in_Ma):
                for c, j_loc_ in enumerate(subset_indices_in_Ma):
                    h_sub[r, c] = h_a[i_loc_, j_loc_]
            
            h_a_i[a][i_task] = h_sub

    # ----------------------------
    # Return results
    # ----------------------------
    results = {
        'U0': U0,         # shape (|M0|, K)
        'U_a': U_a_list,  # list of length = # of assessors
        'eta_a': eta_a_list,
        'h_a': h_a_list,
        'h_a_i': h_a_i
    }
    return results

In [62]:
results = simulate_HPO_with_tasks(
    M0=M0,
    assessors=assessors,
    M_a_list=M_a_list,
    O_a_i_dict=O_a_i_dict,
    alpha=alpha,         # Not None
    K=2,
    rho=0.3,
    tau=0.8,
    random_seed=2025     # Global random seed
)

In [63]:
print("\n--- Global U^(0) ---")
print(results['U0'])

for idx, a in enumerate(assessors):
    print(f"\nAssessor {a}")
    print("  U^(a):\n", results['U_a'][idx])
    print("  eta^(a):\n", results['eta_a'][idx])
    print("  h^(a) adjacency:\n", results['h_a'][idx])

    # Show restricted partial orders
    for i_task, O_subset in enumerate(O_a_i_dict[a]):
        print(f"  Task {i_task}, O_{a,i_task} = {O_subset}")
        print("    restricted adjacency:\n", results['h_a_i'][a][i_task])


--- Global U^(0) ---
[[-0.3230209  -0.46351352]
 [ 1.80666393 -0.03128583]
 [-0.37366743 -0.69472578]
 [ 1.23964294 -0.59862485]
 [ 0.45811139 -0.46849712]
 [-0.4692466   1.25485964]]

Assessor 1
  U^(a):
 [[ 0.80685359  0.71291742]
 [ 0.36260816 -0.69588587]
 [ 1.27601961  1.0781068 ]]
  eta^(a):
 [[ 1.94576989  1.8029057 ]
 [ 1.11223189 -0.04618557]
 [ 2.24013954  1.88785393]]
  h^(a) adjacency:
 [[0 1 0]
 [0 0 0]
 [1 1 0]]
  Task 0, O_(1, 0) = [0, 2, 4]
    restricted adjacency:
 [[0 1 0]
 [0 0 0]
 [1 1 0]]
  Task 1, O_(1, 1) = [0, 2]
    restricted adjacency:
 [[0 1]
 [0 0]]

Assessor 2
  U^(a):
 [[ 1.71805525  0.43724394]
 [-0.74210301 -0.79156413]
 [ 0.8536984  -0.14923463]]
  eta^(a):
 [[ 2.92720027  0.71149668]
 [-0.08796545 -0.13200557]
 [ 1.61892356  0.29915039]]
  h^(a) adjacency:
 [[0 1 1]
 [0 0 0]
 [0 1 0]]
  Task 0, O_(2, 0) = [1, 2, 5]
    restricted adjacency:
 [[0 1]
 [0 0]]
  Task 1, O_(2, 1) = [1, 3]
    restricted adjacency:
 [[0 1]
 [0 0]]
  Task 2, O_(2, 2) = [1,